<a href="https://colab.research.google.com/github/jopapo/nuinvest_options_irpf/blob/main/A%C3%A7%C3%B5es_NuInvest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Isso vai instalar a biblioteca de processamento das notas de negociação em PDF e instalar os módulos necessários ao algoritmo

In [ ]:
# Not installed libs

!pip install pdfminer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 25.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.9 MB/s eta 0:00:00
  Created wheel for pdfminer: filename=pdfminer-20191125-py3-none-any.whl size=6140061 sha256=cd049452cce2b6c9f2367c8e2e941e84a79412c5b9219cb8921b949759d54621
  Stored in directory: /root/.cache/pip/wheels/d1/aa/48/370f83a970d62355a2a47d2d640094a64eea932c22edae1891
Successfully built pdfminer


In [ ]:
# Requirements

import requests
from pathlib import Path
from getpass import getpass
import os
import re
from openpyxl import load_workbook

Isso aqui vai setar a autenticação.

Observação: Ele não autentica. Você deve autenticar na sua conta nuinvest e obter o bearer token usando o dev tools do seu navegador.

In [ ]:
# You should login to your nuinvest account and get the bearer token from the request
access_token = getpass('bearer token')

session = requests.Session()
session.headers.update({'Authorization': f"Bearer {access_token}"})


year2022
bearer token··········


In [92]:
# Variables
year = input('year')

#%rm -rf /content/2021
%cd /content

%mkdir -p {year}
%cd {year}

year2022
/content
/content/2022


Este resumo é interessante mas faltam informações como CNPJ da empresa e taxa. É bom pra fazer uma prova real.

In [93]:
# Get Invoices Pdfs

def get_summary():

  response = session.get(f"https://www.nuinvest.com.br/api/gringott/tradingSummary/1?startDate={year}-01-01&endDate={year}-12-31")
  response.raise_for_status()

  print(response)

  trades = []
  if response.status_code == 204:
    print(f'No data period')
  else:
    trades = response.json()['value']['statements']
  
  totals = {}
  for trade in trades:
    total = totals.get(trade['ticker'])
    if not total:
      total = {
          'sellQuantity': 0,
          'buyQuantity': 0,
          'totalBuyValue': 0.0
      }
    total['sellQuantity'] = total['sellQuantity'] + trade['sellQuantity']
    total['buyQuantity'] = total['buyQuantity'] + trade['buyQuantity']
    total['totalBuyValue'] = total['totalBuyValue'] + trade['buyValue']
    totals[trade['ticker']] = total
    #print(trade)

  return totals

def get_companies():
  filename = Path("Companies.xlsx")
  if not filename.exists():
    response = requests.get("https://www.infomoney.com.br/wp-content/uploads/2022/03/Planilha-CNPJ-das-empresas-da-B3.xlsx")
    response.raise_for_status()  
    filename.write_bytes(response.content)

  wb = load_workbook(filename=filename.name)
  sheet = wb['Principal']
  names = {}
  for i in range(14, sheet.max_row + 1):
    names[sheet[i][1].value] = {
        'cnpj': sheet[i][4].value,
        'name': sheet[i][3].value,
        'irpf_code': 31
    }

  #print(names)
  return names


summary = get_summary()

companies = get_companies()

def get_company(ticker):
  company = companies.get(ticker[:4])
  if not company:
    # Se não tiver ação, entendemos que é FII
    response = requests.get(f"https://informederendimentos.com/consulta/cnpj-{ticker}/")
    response.raise_for_status()
    txt = response.text

    company = {
        'irpf_code': 73 # 73 - Fundos de investimento imobiliário. (ou 26 - outro)
    } 

    matches = re.search(r"\d{2}\.\d{3}\.\d{3}\/\d{4}\-\d{2}", txt)
    company['cnpj'] = matches.group(0)

    matches = re.search(r"\bNome empresarial: \b(.+)\.", txt)
    company['name'] = matches.group(1)

    #: Empresa de São Paulo/SP fundada em 05/11/2015. Sua atividade principal é fundos de investimento imobiliários. Nome empresarial: FUNDO DE INVESTIMENTO IMOBILIARIO GREEN TOWERS.
    companies[ticker[:4]] = company

  return company

for ticker, values in summary.items():
  average_price = values['totalBuyValue'] / values['buyQuantity']
  company = get_company(ticker)
  ownedQuantity = values['buyQuantity'] - values['sellQuantity']
  print(ticker, 'averageBuyValue:', round(average_price, 2), 
        'ownedQuantity:', ownedQuantity,
        'ownedValue:', round(ownedQuantity * average_price, 2), company)


# Ações: você pode utilizar o preço médio de compra e o código 31 - Ações.
# FIIs: você pode utilizar o preço médio de compra e o código 73 - Fundos de investimento imobiliário. (ou 26 - outro)
# ETFs: você pode utilizar o preço médio de compra e o código 74 - Fundos de investimento de índice de mercado.
# Juros e dividendos creditados e não pagos: você pode utilizar o código 99 - Outros bens e direitos.

<Response [200]>
MGLU3F averageBuyValue: 3.7 ownedQuantity: 364 ownedValue: 1345.39 {'cnpj': '47960950000121', 'name': 'MAGAZINE LUIZA S.A.', 'irpf_code': 31}
ABEV3F averageBuyValue: 14.66 ownedQuantity: 46 ownedValue: 674.5 {'cnpj': '07526557000100', 'name': 'AMBEV S.A.', 'irpf_code': 31}
ITUB4F averageBuyValue: 23.1 ownedQuantity: 10 ownedValue: 231.0 {'cnpj': '60872504000123', 'name': 'ITAU UNIBANCO HOLDING S.A.', 'irpf_code': 31}
LVBI11 averageBuyValue: 100.85 ownedQuantity: 15 ownedValue: 1512.77 {'irpf_code': 73, 'cnpj': '30.629.603/0001-18', 'name': 'FUNDO DE INVESTIMENTO IMOBILIARIO &#8211; VBI LOGISTICO'}
MGLU3 averageBuyValue: 2.81 ownedQuantity: 400 ownedValue: 1125.0 {'cnpj': '47960950000121', 'name': 'MAGAZINE LUIZA S.A.', 'irpf_code': 31}
GTWR11 averageBuyValue: 69.75 ownedQuantity: 6 ownedValue: 418.53 {'irpf_code': 73, 'cnpj': '23.740.527/0001-58', 'name': 'FUNDO DE INVESTIMENTO IMOBILIARIO GREEN TOWERS'}


Esse trecho vai baixar todas as notas de negociação (pdf) da NuInvest localmente.

Observação: Não testado com FI e RV.

In [94]:
# Get Invoices Pdfs

def get_invoices(prefix):

  response = session.get(f"https://www.nuinvest.com.br/api/gringott/invoices/1/{prefix}?startDate={year}-01-01&endDate={year}-12-31")
  response.raise_for_status()

  print(prefix, response)

  invoices = []
  if response.status_code == 204:
    print(f'No data for {prefix}')
  else:
    invoices = response.json()['value']['invoices']
  
  for invoice in invoices:
    params_values = {key:val for (key,val) in invoice.items() if key in ['invoiceNumber', 'custodyId', 'date']}
    # FI não tem custodyId
    # RV tem o data e não tem custodyid
    # Não testado com TP e PS pq eu não tinha papéis pra isso

    response = session.get(f"https://www.nuinvest.com.br/api/gringott/invoices/report/1/{prefix}", params=params_values, stream=True)
    response.raise_for_status()

    filename = Path(f"Invoice_{prefix}_" + '_'.join(str(x) for x in params_values.values()) + '.pdf')
    filename.write_bytes(response.content)

papers = ['TD', 'TP', 'FI', 'RV', 'PS']
for paper in papers:
  get_invoices(paper)



TD <Response [200]>
TP <Response [204]>
No data for TP
FI <Response [204]>
No data for FI
RV <Response [200]>
PS <Response [204]>
No data for PS


Esse trecho minera os PDFs e transforma em texto para ser mais fácil interpretar os dados.

In [95]:
%%bash
for f in ./*.pdf
do
 pdf2txt.py -o $f.txt $f
done

Aqui lemos os textos e sumarizamos o que deve ser declarado e onde no IRPF.

Importante: Não nos responsabilizamos pela corretude dessas informações. Use por sua conta e risco.

In [96]:
# FI = fundo de investimento
# TD = tesouro direto
# TP = tesouro privado
# RV = B3 ações/opções
# PS = títulos públicos

data = {}

for entry in os.scandir('.'):
    if entry.is_file() and entry.name.endswith('.txt'):
      txt = Path(entry).read_text()
      #print(entry)

      if entry.name.startswith('Invoice_FI_'):
        matches = re.search(r"\n\bCNPJ Fundo\n(.+)\b", txt)
        cnpj = matches.group(1)

        cumulated = data.get(cnpj)
        if not cumulated:
          cumulated = {}

          matches = re.search(r"\n\bFundo\n(.+)\b", txt)
          cumulated['name'] = matches.group(1)
          
          cumulated['buy_quantity'] = 0
          cumulated['total_buy_value'] = 0
          cumulated['sell_quantity'] = 0
          cumulated['irpf_code'] = 73 # FIIs: você pode utilizar o preço médio de compra e o código 73 - Fundos de investimento imobiliário.

        matches = re.search(r"\n\bQuantidade de cotas\n(.+)\b", txt)
        share_qty = float(matches.group(1).replace('.','').replace(',','.'))

        #matches = re.search(r"\n\bValor da Cota\n(.+)\b", txt)
        #share_value = matches.group(1)
        
        matches = re.search(r"\n\bValor da Operação\n(.+)\b", txt)
        operation_value = float(matches.group(1).replace('.','').replace(',','.').replace('R$ ',''))

        matches = re.search(r"\n\bNota de\n(.+)\b", txt)
        trade_type = matches.group(1)
        if trade_type == 'APLICAÇÃO':
          cumulated['buy_quantity'] = cumulated['buy_quantity'] + share_qty
          cumulated['total_buy_value'] = cumulated['total_buy_value'] + operation_value
        else:
          cumulated['sell_quantity'] = cumulated['sell_quantity'] + share_qty

        cumulated['average_buy_value'] = cumulated['total_buy_value'] / cumulated['buy_quantity']
        cumulated['owned_total'] = cumulated['average_buy_value'] * (cumulated['buy_quantity'] - cumulated['sell_quantity'])

        data[cnpj] = cumulated

      elif entry.name.startswith('Invoice_TD_'):
        matches = re.search(r"\n\bTipo\n(.+)\b", txt)
        
        cnpj = '62.169.875/0001-79' # Tesouro Direto é a própria corretora

        cumulated = data.get(cnpj)
        if not cumulated:
          cumulated = {}

          matches = re.search(r"\n\bTítulo\n(.+)\b", txt)
          cumulated['name'] = matches.group(1)
          
          cumulated['buy_quantity'] = 0
          cumulated['total_buy_value'] = 0
          cumulated['sell_quantity'] = 0
          cumulated['irpf_code'] = 45 # 45 – Aplicação de renda fixa (CDB, RDB e outros)

        matches = re.search(r"\n\bQuantidade\n(.+)\b", txt)
        share_qty = float(matches.group(1).replace('.','').replace(',','.'))

        #matches = re.search(r"\n\bValor 1 título\n(.+)\b", txt)
        #share_value = matches.group(1)
        
        matches = re.search(r"\n\bValor Total\n(.+)\b", txt)
        operation_value = float(matches.group(1).replace('.','').replace(',','.').replace('R$ ',''))

        matches = re.search(r"\n\bTipo\n(.+)\b", txt)
        trade_type = matches.group(1)
        if trade_type == 'Compra':
          cumulated['buy_quantity'] = cumulated['buy_quantity'] + share_qty
          cumulated['total_buy_value'] = cumulated['total_buy_value'] + operation_value
        else:
          cumulated['sell_quantity'] = cumulated['sell_quantity'] + share_qty

        try:
          cumulated['average_buy_value'] = cumulated['total_buy_value'] / cumulated['buy_quantity']
          cumulated['owned_total'] = cumulated['average_buy_value'] * (cumulated['buy_quantity'] - cumulated['sell_quantity'])
        except ZeroDivisionError:
          pass

        data[cnpj] = cumulated

      elif entry.name.startswith('Invoice_RV_'):

        # Isso vai ser pego pelo bloco anterior.
        pass

        # regex = r"(\bFRACIONARIO\b|\bVISTA\b)"
        # matches = re.finditer(regex, txt, re.MULTILINE)
        # for matchNum, match in enumerate(matches, start=1):
        #   print(match)
        #   data[matchNum] = {'type': match.group(0)}
        #   word = re.compile("\w+", re.MULTILINE)
        #   next_word = word.match(txt, match.endpos)
        #   print(next_word)
        #   data[matchNum]['option'] = next_word.group(0)

      else:
        print('Não implementado:', entry.name)

      
import json
print(json.dumps(data, indent=4, sort_keys=True))

{
    "62.169.875/0001-79": {
        "buy_quantity": 0,
        "irpf_code": 45,
        "name": "Tesouro Selic 2027",
        "sell_quantity": 1.4299999999999997,
        "total_buy_value": 0
    }
}
